# First make sure you have ran Scraper.py first to generate data!

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Import the necessary libraries

In [5]:
import pandas as pd
import re
from datetime import datetime, timedelta
import json
import yfinance as yf
import ast
import math
from scipy.stats import norm
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import numpy as np

from qiskit import QuantumCircuit
from qiskit.algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit_aer.primitives import Sampler
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit.utils import QuantumInstance

from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

from concurrent.futures import ThreadPoolExecutor
import time
import random

# Save the IBMid Account and then Load it

In [11]:
API_key = ""

In [6]:
IBMQ.save_account(API_key, overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
available_backends = provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and not x.configuration().simulator)
backend = least_busy(available_backends)

In [7]:
backend

<IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>

# Data Processing

### Enter the correct fileName

In [8]:
fileName = 'option_data_2023_08_14.csv'

match = re.search(r'(\d{4})_(\d{2})_(\d{2})', fileName)
year, month, day = map(int, match.groups())

pDay = datetime(year, month, day, 21, 0)
data = pd.read_csv(fileName, parse_dates=['expiration_date'], date_parser=lambda x: datetime.strptime(x, '%d/%m/%Y'))

In [9]:
tickerArray = data['ticker'].unique().tolist()
tickerDict = {ticker: [] for ticker in tickerArray}

def get_closing_price(ticker_symbol, date):
    stock = yf.Ticker(ticker_symbol)
    hist = stock.history(start=date)
    return round(hist['Close'].iloc[0],2)

try:
    with open(f"ticker_prices_{year}_{month}_{day}.json", "r") as json_file:
        tickerDict = json.load(json_file)
        print("File found!")
        
except FileNotFoundError:
    print('File not found. Creating it now!')
    for t in tickerArray:    
        date = f"{year}-{month}-{day}"
        tickerDict[t].append(get_closing_price(t, date))
    with open (f"ticker_prices_{year}_{month}_{day}.json", "w") as json_file:
        json.dump(tickerDict, json_file)

File found!


In [10]:
data

,strike,lastPrice,bid,ask,impliedVolatility,ticker,expiration_date
0,170.0,9.75,9.55,9.90,0.384772,AAPL,2023-08-18
1,172.5,7.10,7.10,7.45,0.319343,AAPL,2023-08-18
2,175.0,4.93,4.80,5.00,0.246590,AAPL,2023-08-18
3,177.5,2.90,2.85,2.93,0.213631,AAPL,2023-08-18
4,180.0,1.35,1.34,1.35,0.191414,AAPL,2023-08-18
...,...,...,...,...,...,...,...
1833,42.5,2.15,2.13,2.19,0.287849,WFC,2023-09-15
1834,45.0,0.73,0.72,0.74,0.239021,WFC,2023-09-15
1835,47.5,0.15,0.15,0.17,0.226570,WFC,2023-09-15
1836,50.0,0.04,0.04,0.05,0.248054,WFC,2023-09-15


In [11]:
data['expiration_date'] += timedelta(hours=21)
data['time_to_expire_years'] = (data['expiration_date'] - pDay).dt.days / 365
data['market_price'] = round((data['bid'] + data['ask']) / 2,2)

del data['lastPrice'] 
del data['bid']
del data['ask']
del data['expiration_date']

data.reset_index(inplace=True)
del data['index']

In [12]:
df = pd.DataFrame()


for ticker_symbol in tickerArray:
    current_price = tickerDict[ticker_symbol][0]
    
    unique_expirations = data[data['ticker'] == ticker_symbol]['time_to_expire_years'].unique()
    
    for expiration in unique_expirations:
        filtered_strikes = data[(data['ticker'] == ticker_symbol) & 
                              (data['time_to_expire_years'] == expiration) & 
                              (data['strike'] < current_price)]
        
        top_10_strikes = filtered_strikes.sort_values(by='strike', ascending=False).head(10)
        
        df = pd.concat([df, top_10_strikes])

In [13]:
df.reset_index(inplace=True)
del df['index']

# The raw data will be stored in "df" - samples will be sectioned from this DataFrame

In [14]:
df

,strike,impliedVolatility,ticker,time_to_expire_years,market_price
0,177.5,0.213631,AAPL,0.010959,2.89
1,175.0,0.246590,AAPL,0.010959,4.90
2,172.5,0.319343,AAPL,0.010959,7.28
3,170.0,0.384772,AAPL,0.010959,9.73
4,177.5,0.217781,AAPL,0.030137,3.85
...,...,...,...,...,...
980,30.0,0.751956,WFC,0.068493,13.82
981,42.5,0.287849,WFC,0.087671,2.16
982,40.0,0.357428,WFC,0.087671,4.20
983,37.5,0.473638,WFC,0.087671,6.55


# Research

In [15]:
import concurrent.futures

from qiskit import Aer

backend = Aer.get_backend('qasm_simulator')

## Problem Builder - Single Option

In [16]:
# https://qiskit.org/ecosystem/finance/tutorials/03_european_call_option_pricing.html

def problem_builder(params):
    
    num_uncertainty_qubits = 3
    
    r = 0.0527
    
    K, vol, ticker, T = params

    mu = (r - 0.5 * vol**2) * T + np.log(tickerDict[ticker])
    sigma = vol * np.sqrt(T)
    mean = np.exp(mu + sigma**2 / 2)
    variance = (np.exp(sigma**2) - 1) * np.exp(2* mu + sigma**2)
    stddev = np.sqrt(variance)
    low = np.maximum(0, mean-3*stddev)
    high = mean + 3 * stddev

    uncertainty_model = LogNormalDistribution(num_uncertainty_qubits, mu=mu, sigma = sigma**2, bounds = (low, high))

    c_approx = 0.25
    breakpoints = [low, K]
    slopes = [0, 1]
    offsets = [0, 0]
    f_min = 0
    f_max = high - K
    european_call_objective = LinearAmplitudeFunction(
        num_uncertainty_qubits,
        slopes,
        offsets,
        domain=(low, high),
        image=(f_min, f_max),
        breakpoints=breakpoints,
        rescaling_factor=c_approx,
    )


    num_qubits = european_call_objective.num_qubits
    european_call = QuantumCircuit(num_qubits)
    european_call.append(uncertainty_model, range(num_uncertainty_qubits))
    european_call.append(european_call_objective, range(num_qubits))

    problem = EstimationProblem(
        state_preparation=european_call,
        objective_qubits=[3],
        post_processing=european_call_objective.post_processing,
    )

    
    return problem
        

## IQAE Configuration Setup - Single Stock (check https://quantum-computing.ibm.com/jobs to see if it has ran successfully)

In [17]:
qi = QuantumInstance(backend, shots=4000)
epsilon = 0.01
alpha = 0.05
ae = IterativeAmplitudeEstimation(epsilon_target=epsilon, alpha=alpha, quantum_instance=qi)

In [18]:
single_stock = (df.iloc[0]['strike'], df.iloc[0]['impliedVolatility'], df.iloc[0]['ticker'], df.iloc[0]['time_to_expire_years'])

In [19]:
problem = problem_builder(single_stock)

start_time = time.time()
result = ae.estimate(problem)
end_time = time.time()

print(result.estimation_processed)

[2.88099495]


In [20]:
end_time - start_time

1.5081892013549805

### Parameter Tuning

### Taking 100 option contracts

In [23]:
import numpy as np
import pandas as pd
import time
import random
from concurrent.futures import ThreadPoolExecutor

c_approx_values = np.round(np.arange(0.1, 0.525, 0.025), 3)
epsilon_target_values = np.round(np.arange(0.01, 0.325, 0.025), 3)
shots_values = np.round(np.arange(100, 8200, 200), 3)


sampled_rows = df.sample(100, replace=False)

arr = []
for _, row in sampled_rows.iterrows():
    arr.append((row['strike'], row['impliedVolatility'], row['ticker'],
                row['time_to_expire_years'], row['market_price']))

lowest_cost = float('inf')
params = []
parameter_tuning = []

In [24]:
arr

[(555.0, 0.207527456, 'COST', 0.010958904109589041, 8.9),
 (57.5, 0.229988169, 'KO', 0.08767123287671233, 3.78),
 (290.0, 0.390631094, 'META', 0.049315068493150684, 20.33),
 (140.0, 0.29834686, 'AMZN', 0.049315068493150684, 4.05),
 (128.0, 0.319831021, 'GOOG', 0.030136986301369864, 4.42),
 (152.5, 0.178230874, 'PG', 0.030136986301369864, 3.97),
 (147.0, 0.275886147, 'PG', 0.030136986301369864, 9.1),
 (105.0, 0.242927493, 'MRK', 0.0684931506849315, 5.0),
 (550.0, 0.212623841, 'COST', 0.049315068493150684, 16.9),
 (325.0, 0.283332362, 'HD', 0.049315068493150684, 10.92),
 (170.0, 0.284186846, 'JNJ', 0.010958904109589041, 4.15),
 (420.0, 0.34210863, 'NFLX', 0.049315068493150684, 17.25),
 (86.0, 0.286139951, 'DIS', 0.0684931506849315, 4.25),
 (29.5, 0.369146934, 'BAC', 0.010958904109589041, 1.48),
 (385.0, 0.239753696, 'MA', 0.049315068493150684, 16.62),
 (155.0, 0.148690154, 'PG', 0.010958904109589041, 1.44),
 (195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38),
 (420.0, 0.346014841, 'NF

### Problem Builder

In [25]:
def problem_builder(params, c_approx):
    num_uncertainty_qubits = 3
    
    r = 0.0527
    
    K, vol, ticker, T, _ = params

    mu = (r - 0.5 * vol**2) * T + np.log(tickerDict[ticker])
    sigma = vol * np.sqrt(T)
    mean = np.exp(mu + sigma**2 / 2)
    variance = (np.exp(sigma**2) - 1) * np.exp(2* mu + sigma**2)
    stddev = np.sqrt(variance)
    low = np.maximum(0, mean-3*stddev)
    high = mean + 3 * stddev

    uncertainty_model = LogNormalDistribution(num_uncertainty_qubits, mu=mu, sigma = sigma**2, bounds = (low, high))

    breakpoints = [low, K]
    slopes = [0, 1]
    offsets = [0, 0]
    f_min = 0
    f_max = high - K
    european_call_objective = LinearAmplitudeFunction(
        num_uncertainty_qubits,
        slopes,
        offsets,
        domain=(low, high),
        image=(f_min, f_max),
        breakpoints=breakpoints,
        rescaling_factor=c_approx,
    )


    num_qubits = european_call_objective.num_qubits
    european_call = QuantumCircuit(num_qubits)
    european_call.append(uncertainty_model, range(num_uncertainty_qubits))
    european_call.append(european_call_objective, range(num_qubits))

    problem = EstimationProblem(
        state_preparation=european_call,
        objective_qubits=[3],
        post_processing=european_call_objective.post_processing,
    )

    
    return problem    

#### Performance Evaluation - check cost function here if needed

In [26]:
def evaluate_parameters(data):
    metrics, c_approx, epsilon_target, shots = data
    print(f"Starting evaluation for c_approx={c_approx}, epsilon_target={epsilon_target}, shots={shots}...")    
    problem = problem_builder(metrics, c_approx)
    qi = QuantumInstance(backend=backend, shots=shots)
    qae = IterativeAmplitudeEstimation(epsilon_target=epsilon_target, alpha=0.05, quantum_instance=qi)

    start_time = time.time()
    try:
        result = qae.estimate(problem)
    except AttributeError as e:
        if "NoneType" in str(e):
            print(f"Encountered the NoneType error for c_approx={c_approx}, epsilon_target={epsilon_target}, shots={shots}. Skipping this set of parameters.")
            return {
                'p': metrics,
                'c_approx': c_approx,
                'epsilon_target': epsilon_target,
                'shots': shots,
                'accuracy_error': float('inf'),
                'computation_time': float('inf'),
                'cost': float('inf')
            }
        else:
            raise e

    end_time = time.time()

    computation_time = end_time - start_time
    accuracy_error = np.abs(result.estimation_processed - metrics[-1])
    cost_function =  computation_time + 1.2 * accuracy_error

    print(f"Completed evaluation for c_approx={c_approx}, epsilon_target={epsilon_target}, shots={shots}.")

    return {
        'p': metrics,
        'c_approx': c_approx,
        'epsilon_target': epsilon_target,
        'shots': shots,
        'accuracy_error': float(accuracy_error),
        'computation_time': computation_time,
        'cost': float(cost_function)
    }


### Random Grid Search

In [ ]:
NUM_TRIALS = 100
tasks = []

for metrics in arr:
    for _ in range(NUM_TRIALS):
        c_approx = random.choice(c_approx_values)
        epsilon_target = random.choice(epsilon_target_values)
        shots = random.choice(shots_values)
        tasks.append((metrics, c_approx, epsilon_target, shots))

print("Starting parameter tuning...")

with ThreadPoolExecutor() as executor:
    results = list(executor.map(evaluate_parameters, tasks))

best_parameters_by_problem = {}

for result in results:
    p = result['p']
    current_cost = best_parameters_by_problem.get(p, {}).get('cost', float('inf'))

    if result['cost'] < current_cost:
        best_parameters_by_problem[p] = result

best_params_list = list(best_parameters_by_problem.values())

parameter_tuning_df = pd.DataFrame(best_params_list)
all_results_df = pd.DataFrame(results)

print("Parameter tuning completed. Processing results...")
print("All tasks completed!")

In [211]:
all_results_df.to_csv('paramter_tuning.csv')

## Epilson Target

In [135]:
arr

[(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35),
 (225.0, 0.303473762, 'V', 0.0684931506849315, 19.27),
 (150.0, 0.231452998, 'CVX', 0.0684931506849315, 14.12),
 (35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3),
 (60.0, 0.257819922, 'BMY', 0.010958904109589041, 1.72),
 (390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85),
 (532.5, 0.244514391, 'LLY', 0.010958904109589041, 8.8),
 (520.0, 0.262580616, 'TMO', 0.08767123287671233, 28.3),
 (95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92),
 (490.0, 0.23154455, 'UNH', 0.08767123287671233, 26.38),
 (35.0, 0.258796475, 'PFE', 0.08767123287671233, 1.68),
 (255.0, 0.233345069, 'DHR', 0.030136986301369864, 5.35),
 (70.0, 0.550785742, 'DIS', 0.08767123287671233, 19.25),
 (128.0, 0.319831021, 'GOOG', 0.030136986301369864, 4.42),
 (300.0, 0.311042046, 'MSFT', 0.049315068493150684, 24.92),
 (45.0, 0.483403604, 'CSCO', 0.08767123287671233, 9.15),
 (510.0, 0.343024148, 'ADBE', 0.0684931506849315, 24.72),
 (385.0, 0.232124378, 'MA',

### Grid Search

In [136]:
import time
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import random

tasks = []
epsilon_target_values = np.round(np.arange(0.01, 0.325, 0.02), 3)

for metrics in arr:
    for e in epsilon_target_values:
        c_approx = 0.25
        shots = 10000
        tasks.append((metrics, c_approx, e, shots))

print("Starting parameter tuning...")

with ThreadPoolExecutor() as executor:
    results = list(executor.map(evaluate_parameters, tasks))

best_parameters_by_problem = {}

for result in results:
    p = result['p']
    current_cost = best_parameters_by_problem.get(p, {}).get('cost', float('inf'))

    if result['cost'] < current_cost:
        best_parameters_by_problem[p] = result

epsil_list = list(best_parameters_by_problem.values())

epsilon_tuning_df = pd.DataFrame(best_params_list)
all_results_df = pd.DataFrame(results)

print("Parameter tuning completed. Processing results...")
print("All tasks completed!")

Starting parameter tuning...
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.03, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.05, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.07, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.03, shots=10000.
Completed evaluation for c_approx=0.25, epsilon_target=0.05, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_appro

Completed evaluation for c_approx=0.25, epsilon_target=0.21, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.23, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.29, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.13, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.25, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.15, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.31, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.17, shots=10000...
(35.0, 0.266608896, 'PFE', 0.0301369863

Completed evaluation for c_approx=0.25, epsilon_target=0.15, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.27, shots=10000...
(390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85)
Completed evaluation for c_approx=0.25, epsilon_target=0.05, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.29, shots=10000...
(390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85)
Completed evaluation for c_approx=0.25, epsilon_target=0.19, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.31, shots=10000...
(390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85)
Completed evaluation for c_approx=0.25, epsilon_target=0.27, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000...
(532.5, 0.244514391, 'LLY', 0.010958904109589041, 8.8)
Completed evaluation for c_approx=0.25, epsilon_target=0.17, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.03, shots=10000...
(532.5, 0.244514391, 'LLY', 0.01095890

Completed evaluation for c_approx=0.25, epsilon_target=0.29, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.15, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)
Completed evaluation for c_approx=0.25, epsilon_target=0.07, shots=10000.Completed evaluation for c_approx=0.25, epsilon_target=0.23, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.17, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)

Starting evaluation for c_approx=0.25, epsilon_target=0.19, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)
Completed evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.21, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)
Completed evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.23, shots=10000...
(95.0, 0.300788242, 'XOM', 0.087671

Completed evaluation for c_approx=0.25, epsilon_target=0.23, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.03, shots=10000...
(255.0, 0.233345069, 'DHR', 0.030136986301369864, 5.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.03, shots=10000.Completed evaluation for c_approx=0.25, epsilon_target=0.25, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.05, shots=10000...
(255.0, 0.233345069, 'DHR', 0.030136986301369864, 5.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.07, shots=10000...
(255.0, 0.233345069, 'DHR', 0.030136986301369864, 5.35)

Starting evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000...
(255.0, 0.233345069, 'DHR', 0.030136986301369864, 5.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000...
(255.0, 0.233345069, 'DHR', 0.0

...

Completed evaluation for c_approx=0.25, epsilon_target=0.13, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.03, shots=10000...
(68.0, 0.213875049, 'NEE', 0.0684931506849315, 1.78)
Completed evaluation for c_approx=0.25, epsilon_target=0.15, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.05, shots=10000...
(68.0, 0.213875049, 'NEE', 0.0684931506849315, 1.78)
Completed evaluation for c_approx=0.25, epsilon_target=0.27, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.07, shots=10000...
(68.0, 0.213875049, 'NEE', 0.0684931506849315, 1.78)
Completed evaluation for c_approx=0.25, epsilon_target=0.21, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000...
(68.0, 0.213875049, 'NEE', 0.0684931506849315, 1.78)
Completed evaluation for c_approx=0.25, epsilon_target=0.23, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000...
(68.0, 0.213875049, 'NEE', 0.06849315068493

Completed evaluation for c_approx=0.25, epsilon_target=0.31, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.21, shots=10000...
(415.0, 0.358008129, 'NFLX', 0.0684931506849315, 22.88)
Completed evaluation for c_approx=0.25, epsilon_target=0.21, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.23, shots=10000...
(415.0, 0.358008129, 'NFLX', 0.0684931506849315, 22.88)
Completed evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.25, shots=10000...
(415.0, 0.358008129, 'NFLX', 0.0684931506849315, 22.88)
Completed evaluation for c_approx=0.25, epsilon_target=0.05, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.27, shots=10000...
(415.0, 0.358008129, 'NFLX', 0.0684931506849315, 22.88)
Completed evaluation for c_approx=0.25, epsilon_target=0.15, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.29, shots=10000...
(415.0, 0.358008129, 'NFLX', 0.

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.07, shots=10000...
(560.0, 0.182503292, 'COST', 0.010958904109589041, 5.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.23, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.09, shots=10000...
(560.0, 0.182503292, 'COST', 0.010958904109589041, 5.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.19, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000...
(560.0, 0.182503292, 'COST', 0.010958904109589041, 5.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.29, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.13, shots=10000...
(560.0, 0.182503292, 'COST', 0.010958904109589041, 5.35)
Completed evaluation for c_approx=0.25, epsilon_target=0.21, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.15, shots=10000...
(560.0, 0.182503292, 'COST'

Completed evaluation for c_approx=0.25, epsilon_target=0.29, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.25, shots=10000...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.03, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.27, shots=10000...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.29, shots=10000...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.05, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.31, shots=10000...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.13, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000...
(105.0, 0.259895096, 'NKE', 0.087671232

Completed evaluation for c_approx=0.25, epsilon_target=0.21, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.11, shots=10000...
(850.0, 0.423620266, 'AVGO', 0.0684931506849315, 39.55)
Completed evaluation for c_approx=0.25, epsilon_target=0.27, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.13, shots=10000...
(850.0, 0.423620266, 'AVGO', 0.0684931506849315, 39.55)
Completed evaluation for c_approx=0.25, epsilon_target=0.25, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.15, shots=10000...
(850.0, 0.423620266, 'AVGO', 0.0684931506849315, 39.55)
Completed evaluation for c_approx=0.25, epsilon_target=0.07, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.17, shots=10000...
(850.0, 0.423620266, 'AVGO', 0.0684931506849315, 39.55)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.19, shots=10000...
(850.0, 0.423620266, 'AVGO', 0.

In [137]:
import pandas as pd

all_results_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [140]:
all_results_df.to_csv('epsilon_target.csv')

### Grid Search

In [141]:
tasks = []
shots_values = np.round(np.arange(100, 8200, 200), 3)


for metrics in arr:
    for s in shots_values:
        c_approx = 0.25
        epsilon = 0.01
        shots = s
        tasks.append((metrics, c_approx, epsilon, s))

print("Starting parameter tuning...")

with ThreadPoolExecutor() as executor:
    results = list(executor.map(evaluate_parameters, tasks))

best_parameters_by_problem = {}

for result in results:
    p = result['p']
    current_cost = best_parameters_by_problem.get(p, {}).get('cost', float('inf'))

    if result['cost'] < current_cost:
        best_parameters_by_problem[p] = result

# Assuming the results still have a best value per problem to be extracted
best_params_list = list(best_parameters_by_problem.values())

# Create DataFrames
shots_tuning_df = pd.DataFrame(best_params_list)
all_results_df = pd.DataFrame(results)

print("Parameter tuning completed. Processing results...")
print("All tasks completed!")


Starting parameter tuning...
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=100...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=300...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=500...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=700...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=900...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=1100...Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=1300...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)

(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.01,

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=1300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=2100...
(225.0, 0.303473762, 'V', 0.0684931506849315, 19.27)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=4700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=2300...
(225.0, 0.303473762, 'V', 0.0684931506849315, 19.27)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=4500.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=2500...
(225.0, 0.303473762, 'V', 0.0684931506849315, 19.27)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=5700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=2700...
(225.0, 0.303473762, 'V', 0.0684931506849315, 19.27)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=5300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=2900...
(225.0, 0.303473762, 'V', 0.0684931506849315, 19.27)


Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=3700...
(150.0, 0.231452998, 'CVX', 0.0684931506849315, 14.12)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=5500.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=3900...
(150.0, 0.231452998, 'CVX', 0.0684931506849315, 14.12)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=6900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=4100...
(150.0, 0.231452998, 'CVX', 0.0684931506849315, 14.12)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=1300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=4300...
(150.0, 0.231452998, 'CVX', 0.0684931506849315, 14.12)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=4500...
(150.0, 0.231452998, 'CVX', 0.0684931506849315

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=1700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=4700...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=4900...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=5700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=5100...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=5300...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=6100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=5500...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=1700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=5500...
(60.0, 0.257819922, 'BMY', 0.010958904109589041, 1.72)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=5700...
(60.0, 0.257819922, 'BMY', 0.010958904109589041, 1.72)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=5900...
(60.0, 0.257819922, 'BMY', 0.010958904109589041, 1.72)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=6100...
(60.0, 0.257819922, 'BMY', 0.010958904109589041, 1.72)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=6300...
(60.0, 0.257819922, 'BMY', 0.0109589041095890

...

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=4700.Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=6900...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)

Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7100...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=6700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7300...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7500...
(195.0, 0.45837944, 'CRM', 0.0684931506849315, 20.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2300.Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=3100.
Starting evaluation for c_approx=0.25, epsilon_targ

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=6900...
(105.0, 0.259895096, 'NKE', 0.08767123287671233, 4.75)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7100...
(105.0, 0.259895096, 'NKE', 0.08767123287671233, 4.75)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=1900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7300...
(105.0, 0.259895096, 'NKE', 0.08767123287671233, 4.75)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7500...
(105.0, 0.259895096, 'NKE', 0.08767123287671233, 4.75)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2500.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7700...
(105.0, 0.259895096, 'NKE', 0.087671232876712

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7700...
(517.5, 0.321173781, 'ADBE', 0.030136986301369864, 14.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=3100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7900...
(517.5, 0.321173781, 'ADBE', 0.030136986301369864, 14.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=8100...
(517.5, 0.321173781, 'ADBE', 0.030136986301369864, 14.38)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=3500.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=100...
(850.0, 0.423620266, 'AVGO', 0.0684931506849315, 39.55)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=300...
(850.0, 0.423620266, 'AVGO', 0.068493

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7700.Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7900...
(850.0, 0.423620266, 'AVGO', 0.0684931506849315, 39.55)

Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=8100...
(850.0, 0.423620266, 'AVGO', 0.0684931506849315, 39.55)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=1900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=100...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=3900.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=300...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2700.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=500...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 

Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7100.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=7900...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=7300.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=8100...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2900.
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=2700.
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=500.
Encountered the NoneType error for c_approx=0.25, epsilon_target=0.01, shots=4500. Skipping this set of parameters.
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=8100.
Encountered the NoneType error for c_approx=0.25, epsilon_target=0.01, shots=3700. Skipping this set of parameters.
Encountered the NoneType error for c_approx=0.25, epsilon_target=0.01, shot

In [142]:
all_results_df.to_csv('shots_results.csv')

### Grid Search

In [145]:
tasks = []
c_approx_values = np.round(np.arange(0.1, 0.525, 0.025), 3)

for metrics in arr:
    for c in c_approx_values:
        epsilon = 0.01
        shots = 10000
        tasks.append((metrics, c, epsilon, shots))

print("Starting parameter tuning...")

with ThreadPoolExecutor() as executor:
    results = list(executor.map(evaluate_parameters, tasks))

best_parameters_by_problem = {}

for result in results:
    p = result['p']
    current_cost = best_parameters_by_problem.get(p, {}).get('cost', float('inf'))

    if result['cost'] < current_cost:
        best_parameters_by_problem[p] = result


best_params_list = list(best_parameters_by_problem.values())

# Create DataFrames
c_tuning_df = pd.DataFrame(best_params_list)
all_results_df = pd.DataFrame(results)

print("Parameter tuning completed. Processing results...")
print("All tasks completed!")

Starting parameter tuning...
Starting evaluation for c_approx=0.1, epsilon_target=0.01, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.125, epsilon_target=0.01, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.15, epsilon_target=0.01, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000...
Starting evaluation for c_approx=0.2, epsilon_target=0.01, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000...
(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35)
Starting evaluation for c_approx=0.275, epsilo

Completed evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.2, epsilon_target=0.01, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.45, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000...
(35.0, 0.266608896, 'PFE', 0.030136986301369864, 1.3)
Completed evaluation for c_approx=0.15, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.275, epsilon_target=0.01, shots=10000...
(35.0, 0.266608896, 'PFE', 0.03013

Completed evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000...
(390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85)
Completed evaluation for c_approx=0.1, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000...
(390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85)
Completed evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.4, epsilon_target=0.01, shots=10000...
(390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85)
Completed evaluation for c_approx=0.45, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.425, epsilon_target=0.01, shots=10000...
(390.0, 0.223762645, 'MA', 0.0684931506849315, 13.85)
Completed evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.45, epsilon_target=0.01, shots=10000...
(390.0, 0.223762645, 'MA', 0.06849315

Completed evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.1, epsilon_target=0.01, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)
Completed evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000.Completed evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.125, epsilon_target=0.01, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)

Starting evaluation for c_approx=0.15, epsilon_target=0.01, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)
Completed evaluation for c_approx=0.3, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000...
(95.0, 0.300788242, 'XOM', 0.08767123287671233, 16.92)
Completed evaluation for c_approx=0.125, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.2, epsilon_target=0.01, shots=10000...
(95.0, 0.300788242, 'XOM', 0.0876

Completed evaluation for c_approx=0.4, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.275, epsilon_target=0.01, shots=10000...
(35.0, 0.258796475, 'PFE', 0.08767123287671233, 1.68)
Completed evaluation for c_approx=0.3, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.3, epsilon_target=0.01, shots=10000...
(35.0, 0.258796475, 'PFE', 0.08767123287671233, 1.68)
Completed evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000...
(35.0, 0.258796475, 'PFE', 0.08767123287671233, 1.68)
Completed evaluation for c_approx=0.275, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000...
(35.0, 0.258796475, 'PFE', 0.08767123287671233, 1.68)
Completed evaluation for c_approx=0.275, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000...
(35.0, 0.258796475, 'PFE', 0.0876712

...

Completed evaluation for c_approx=0.45, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.3, epsilon_target=0.01, shots=10000...
(157.5, 0.197761929, 'CVX', 0.030136986301369864, 6.72)
Completed evaluation for c_approx=0.2, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000...
(157.5, 0.197761929, 'CVX', 0.030136986301369864, 6.72)
Completed evaluation for c_approx=0.5, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000...
(157.5, 0.197761929, 'CVX', 0.030136986301369864, 6.72)
Completed evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000...
(157.5, 0.197761929, 'CVX', 0.030136986301369864, 6.72)
Completed evaluation for c_approx=0.4, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.4, epsilon_target=0.01, shots=10000...
(157.5, 0.197761929, 'CVX', 0.030

Completed evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.5, epsilon_target=0.01, shots=10000...
(35.0, 0.546879531, 'WFC', 0.08767123287671233, 9.0)
Completed evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.1, epsilon_target=0.01, shots=10000...
(75.0, 0.456548403, 'DIS', 0.08767123287671233, 14.32)
Completed evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.125, epsilon_target=0.01, shots=10000...
(75.0, 0.456548403, 'DIS', 0.08767123287671233, 14.32)
Completed evaluation for c_approx=0.3, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.15, epsilon_target=0.01, shots=10000...
(75.0, 0.456548403, 'DIS', 0.08767123287671233, 14.32)
Completed evaluation for c_approx=0.25, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000...
(75.0, 0.456548403, 'DIS', 0.087671

Completed evaluation for c_approx=0.45, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.275, epsilon_target=0.01, shots=10000...
(36.0, 0.225105405, 'PFE', 0.0684931506849315, 0.88)
Completed evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.3, epsilon_target=0.01, shots=10000...
(36.0, 0.225105405, 'PFE', 0.0684931506849315, 0.88)
Completed evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000...
(36.0, 0.225105405, 'PFE', 0.0684931506849315, 0.88)
Completed evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000...
(36.0, 0.225105405, 'PFE', 0.0684931506849315, 0.88)
Completed evaluation for c_approx=0.15, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000...
(36.0, 0.225105405, 'PFE', 0.0684931506

Completed evaluation for c_approx=0.475, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.125, epsilon_target=0.01, shots=10000...
(517.5, 0.321173781, 'ADBE', 0.030136986301369864, 14.38)
Completed evaluation for c_approx=0.5, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.15, epsilon_target=0.01, shots=10000...
(517.5, 0.321173781, 'ADBE', 0.030136986301369864, 14.38)
Completed evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.175, epsilon_target=0.01, shots=10000...
(517.5, 0.321173781, 'ADBE', 0.030136986301369864, 14.38)
Completed evaluation for c_approx=0.5, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.2, epsilon_target=0.01, shots=10000...
(517.5, 0.321173781, 'ADBE', 0.030136986301369864, 14.38)
Completed evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000...
(517.5, 0.321173781,

Completed evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.275, epsilon_target=0.01, shots=10000...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.225, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.3, epsilon_target=0.01, shots=10000...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.125, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.325, epsilon_target=0.01, shots=10000...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.4, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.35, epsilon_target=0.01, shots=10000...
(52.0, 0.295173064, 'CSCO', 0.0684931506849315, 2.74)
Completed evaluation for c_approx=0.15, epsilon_target=0.01, shots=10000.
Starting evaluation for c_approx=0.375, epsilon_target=0.01, shots=10000...
(52.0, 0.295173064, 'CSCO', 0.068493

In [146]:
all_results_df.to_csv('c_approx_results.csv')

# Now go to results_analysis.ipynb file and come back once you have ran all the cells

### Monte Carlo

In [62]:
import numpy as np
from scipy.stats import norm
import time

def monte_carlo_european_call(S0, K, T, r, sigma, n_paths, n_steps):
        
    z = np.random.randn(n_paths // 2, n_steps)
    z = np.concatenate((z, -z), axis=0)
    
    
    S = np.zeros((n_paths, n_steps + 1))
    S[:, 0] = S0
    
    dt = T / n_steps
    for t in range(1, n_steps + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z[:, t-1])
    
    
    payoff = np.maximum(S[:, -1] - K, 0)
    
    
    d1 = (np.log(S0/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    BS_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    control_variate = BS_price + np.mean(payoff - BS_price)
    
    discount_factor = np.exp(-r * T)
    option_price = discount_factor * np.mean(control_variate)
    
    end = time.time()
    
    
    return round(option_price,2)

In [43]:
arr = []
sampled_rows = df.sample(n=20)
for _, row in sampled_rows.iterrows():
    arr.append((row['strike'], row['impliedVolatility'], row['ticker'],
                row['time_to_expire_years'], row['market_price']))

In [44]:
arr

[(155.0, 0.397955239, 'TXN', 0.049315068493150684, 16.35),
 (160.0, 0.167244656, 'CVX', 0.030136986301369864, 4.47),
 (420.0, 0.643100493, 'NVDA', 0.08767123287671233, 42.52),
 (29.0, 0.441411836, 'VZ', 0.0684931506849315, 4.7),
 (515.0, 0.290778577, 'LLY', 0.049315068493150684, 27.22),
 (845.0, 0.460118924, 'AVGO', 0.049315068493150684, 39.2),
 (165.0, 0.339362075, 'TXN', 0.010958904109589041, 6.02),
 (555.0, 0.195656237, 'COST', 0.0684931506849315, 14.75),
 (840.0, 0.426397338, 'AVGO', 0.0684931506849315, 45.15),
 (131.0, 0.240730249, 'GOOG', 0.010958904109589041, 1.9),
 (103.0, 0.238777144, 'ABT', 0.049315068493150684, 4.08),
 (175.0, 0.240547145, 'AAPL', 0.08767123287671233, 7.58),
 (107.0, 0.263190962, 'NKE', 0.030136986301369864, 2.36),
 (29.0, 0.347662773, 'BAC', 0.030136986301369864, 2.04),
 (427.5, 0.305915144, 'NFLX', 0.010958904109589041, 6.15),
 (157.5, 0.310065493, 'WMT', 0.030136986301369864, 4.88),
 (550.0, 0.204048487, 'COST', 0.08767123287671233, 19.98),
 (43.0, 0.3847

In [114]:
def calculate_option_price(opt):
    S0 = tickerDict[opt[2]][0]
    K = opt[0]
    T = opt[3]
    r = 0.0527
    sigma = opt[1]
    n_paths = 100000
    n_steps = math.ceil(T * 365)
    mkt_price = opt[-1]
    
    start = time.time()
    opt_price = monte_carlo_european_call(S0, K, T, r, sigma, n_paths, n_steps)
    computation_time = np.round(time.time() - start, 2)
    accuracy_error = np.round(np.abs(opt_price - mkt_price), 2)
    
    result = {
        'option_contract': opt,
        'option_price': opt_price, 
        'market_price': mkt_price, 
        'computation_time': computation_time, 
        'accuracy_error': accuracy_error
    }
    
    return result

In [115]:
results = list(map(calculate_option_price, arr))
MC_df = pd.DataFrame(results)

In [118]:
MC_df['accuracy_error'].sum()

11.36

# Enter the values from the Cluster Analysis -- parameters with the lowest average accuracy

In [9]:
epsilon_target = 0
shots = 0
c_approx = 0

## Make sure the above values are populated before running

In [10]:
if epsilon_target == 0 or shots == 0 or c_approx == 0:
    raise Exception ("Please enter the values")

In [132]:
provider = IBMQ.get_provider(hub='ibm-q')
available_backends = provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and not x.configuration().simulator)
backend = least_busy(available_backends)

qi = QuantumInstance(backend, shots=int(shots))
epsilon = epsilon_target
alpha = 0.05
ae = IterativeAmplitudeEstimation(epsilon_target=epsilon, alpha=alpha, quantum_instance=qi)

In [133]:
backend

<IBMQBackend('ibm_lagos') from IBMQ(hub='ibm-q', group='open', project='main')>

In [135]:
thread_start_qc = time.time()
with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda element: evaluate_parameters((tuple(element), c_approx, epsilon_target, int(shots))), arr))

thread_end_qc = time.time()

Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluat

FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c6595c43f6a990a6b0e79/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c6595b1be0c1d9ea14089/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request

Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.


FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c659e2a816a06fb3d25e4/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c659db8c12765f7be5d10/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request

Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.


...

Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.


FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c65c20081284cf60abbb8/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c65c2a102d83669a8c285/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request

Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.


FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c65c9c739d8acafc9a2d0/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs/652c65c9c43f6a58356b0e85/jobDataUploaded. Error in runtime client calling method runJob: Error executing POST https://runtime-us-east.quantum-computing.ibm.com/jobs {No request body}, Error code: 2413.'" 
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: "Error submitting job: '400 Client Error: Bad Request

Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.


In [136]:
QC_df = pd.DataFrame(results)

In [137]:
QC_df

,p,c_approx,epsilon_target,shots,accuracy_error,computation_time,cost
0,"(155.0, 0.397955239, TXN, 0.049315068493150684...",0.237347,0.080041,6384,15.704337,19.440411,38.285616
1,"(160.0, 0.167244656, CVX, 0.030136986301369864...",0.237347,0.080041,6384,4.202542,41.344453,46.387503
2,"(420.0, 0.643100493, NVDA, 0.08767123287671233...",0.237347,0.080041,6384,81.777561,14.305713,112.438787
3,"(29.0, 0.441411836, VZ, 0.0684931506849315, 4.7)",0.237347,0.080041,6384,3.675602,35.569751,39.980472
4,"(515.0, 0.290778577, LLY, 0.049315068493150684...",0.237347,0.080041,6384,36.955725,46.474621,90.821491
5,"(845.0, 0.460118924, AVGO, 0.04931506849315068...",0.237347,0.080041,6384,91.827857,68.214271,178.407700
6,"(165.0, 0.339362075, TXN, 0.010958904109589041...",0.237347,0.080041,6384,6.420816,23.748376,31.453356
7,"(555.0, 0.195656237, COST, 0.0684931506849315,...",0.237347,0.080041,6384,29.003310,29.397374,64.201346
8,"(840.0, 0.426397338, AVGO, 0.0684931506849315,...",0.237347,0.080041,6384,96.490282,73.701447,189.489785
9,"(131.0, 0.240730249, GOOG, 0.01095890410958904...",0.237347,0.080041,6384,3.320814,96.783084,100.768061


In [141]:
thread_end_qc - thread_start_qc

98.40564703941345

In [138]:
QC_df.to_csv('multiple_quantum_results.csv')

# Running on simulator

In [101]:
backend = Aer.get_backend('qasm_simulator')

qi = QuantumInstance(backend, shots=int(shots))
epsilon = epsilon_target
alpha = 0.05
ae = IterativeAmplitudeEstimation(epsilon_target=epsilon, alpha=alpha, quantum_instance=qi)

In [119]:
thread_start_time = time.time()
with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda element: evaluate_parameters((tuple(element), c_approx, epsilon_target, int(shots))), arr))
thread_end_time = time.time()

print(thread_end_time - thread_start_time)

Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...

Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Starting evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384...
Completed evaluation for c_approx=0.237347, epsilon_target=0.080041, shots=6384.
Starting evaluation 

In [120]:
QC_sim_df = pd.DataFrame(results)

In [142]:
QC_sim_df

17.12177168350431